In [85]:
! pip install python_tsp

In [86]:
import numpy as np
from collections import deque
import random
import copy
import math
import tensorflow as tf
import keras
from python_tsp.exact import solve_tsp_dynamic_programming
import numpy as np

# Modeling
def QNet(input_shape, output_shape):
  optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
  criterion = tf.losses.MeanSquaredError()
  Qnet = tf.keras.Sequential([
         tf.keras.layers.Dense(units=256, activation='relu', input_shape=(input_shape,)),
         tf.keras.layers.Dense(units=256, activation='relu'),
         tf.keras.layers.Dense(units=output_shape, activation='linear')
        ])
  Qnet.compile(optimizer=optimizer, loss=criterion)

  return Qnet

# 예시 데이터

input_shape = 10  # 예시 입력 차원
output_shape = 10  # 예시 출력 차원
input_data = tf.constant([[1.0] * input_shape], dtype=tf.float32)

# 학습 예시
model = QNet(input_shape, output_shape)
model.fit(input_data, input_data, batch_size=1)

# 예측 예시
output_data = model.predict(input_data)
print(output_data)

1/1 [==============================] - 0s 60ms/step
[[1.2476184  1.18904    1.2200441  1.7473732  1.5378778  1.2865838
  0.94348234 1.2788025  0.799048   1.0779693 ]]


In [87]:
def cal_dist(point1, point2):
    return math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

def TSPDistcost(points):
    num_points = len(points)
    distance_matrix = [[0] * num_points for _ in range(num_points)]

    for i in range(num_points):
        for j in range(i + 1, num_points):
            distance = cal_dist(points[i], points[j])
            distance_matrix[i][j] = distance
            distance_matrix[j][i] = distance
    return distance_matrix

def randcoord(numnodes, maxvalue):
    nodes = np.random.rand(numnodes, 2) * maxvalue
    return nodes

def Findoptsol(distance_matrix):
    permutation, distance = solve_tsp_dynamic_programming(np.array(distance_matrix))
    return permutation, distance

In [88]:
class Env():
    def __init__(self, points):
        self.nodes = points
        self.n_stops = len(points)
        self.dist_matrix = TSPDistcost(points)
        self.start = 0
        self.togo = [x for x in range(1, len(points))]
        self.visited = [0]
        self.done = False

    def reset(self):
        self.start = 0
        self.togo = [x for x in range(1, len(self.nodes))]
        self.visited = [0]
        self.done = False
        return self.start

    def step(self, s, a):
        s_next = a
        re = -self.dist_matrix[s][s_next]
        if s_next in self.togo:
            k = self.togo.index(s_next)
            self.togo.pop(k)
            self.visited.append(s_next)
        if len(self.togo) == 0:
            self.done = True
        return s_next, re, self.done

In [89]:
def create_one_hot_list(length, indices):
    one_hot_list = [0] * length

    for index in indices:
        if 0 <= index < length:
            one_hot_list[index] = 1

    return one_hot_list

def trans(s, visit, length):
    currentnode = np.identity(length)[s]
    visitednodes = create_one_hot_list(length, visit)
    input = np.concatenate([currentnode, visitednodes], axis=0)
    return input

In [90]:
class Agent():
    def __init__(self, states_size, actions_size, buffer_limit=20, batch_size=10, \
                 epsilon_min=0.01, epsilon_decay=0.99, gamma=0.95, lr=0.01, C=10):
        self.states_size = states_size
        self.actions_size = actions_size
        self.batch_size = batch_size
        self.buffer_limit = buffer_limit
        self.epsilon = 1
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.gamma = gamma
        self.lr = lr
        self.C = C
        self.Qnet = QNet(states_size, actions_size)
        self.Tnet = copy.deepcopy(self.Qnet)
        self.buffer = deque(maxlen=buffer_limit)

    def reset_memory(self):
        self.buffer = deque(maxlen=self.buffer_limit)

    def act(self, input):
        input = input.reshape(1,-1)
        q = self.Qnet.predict(input, verbose=0)
        if np.random.rand() > self.epsilon:
            a = np.argmax(q)
        else:
            a = np.random.randint(self.actions_size)
        return a

    def put(self, s, visit, a, r, s_next):
        self.buffer.append((s, visit, a, r, s_next))

    def sample(self):
        mini_batch = random.sample(self.buffer, self.batch_size)
        input_lst, a_lst, r_lst, input_next_lst = [], [], [], []

        for transition in mini_batch:
            s, visit, a, r, s_next = transition

            # s input setting for Q(s,a)
            input = trans(s, visit, self.actions_size)
            input_lst.append(input)

            # s' input setting for max(Q(s_next,a'))
            visit.append(s_next)
            input_next = trans(s_next, visit, self.actions_size)
            input_next_lst.append(input_next)

            a_lst.append(a)
            r_lst.append(r)

        return np.array(input_lst), np.array(a_lst), np.array(r_lst), np.array(input_next_lst)

    def train(self):
        input_lst, a_lst, r_lst, input_next_lst = self.sample()
        print(input_lst.shape, input_next_lst.shape)
        y_hat = np.max(self.Qnet.predict(input_lst.T, verbose=0), axis=1)
        y = r_lst + self.gamma * np.max(self.Tnet.predict(input_next_lst.T, verbose=0), axis=1)
        for i in range(10):
          self.Qnet.fit(y_hat[i], y[i], verbose=0)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def netsynch(self):
        self.Tnet = copy.deepcopy(self.Qnet)

In [91]:
def run_episode(env, agent, num_episodes, verbose=1):
    s = env.reset()
    agent.reset_memory()
    max_step = env.n_stops
    episode_reward = 0

    i = 0
    while i < max_step:
        # Choose an Action
        input_data = trans(s, env.visited, max_step)
        a = agent.act(input_data)  # e-greedy policy

        # Take the action, and get the reward from environment
        if a in env.togo:
            s_next, r, done = env.step(s, a)

            if verbose:
                print(s_next, r, done)

            # Update Model
            # Experience Replay
            # put sample
            agent.put(s, env.visited, a, r, s_next)
            # train Qnet(PredictedNetwork) using a batch of samples in the buffer
            if len(agent.buffer) > agent.batch_size:
               agent.train()

            # TargetNetwork synchronizing with PredictionNetwork
            if num_episodes % agent.C == 0 and num_episodes != 0:
                agent.netsynch()

            # Update the caches
            episode_reward += r
            s = s_next
        else:
            r = -500
            agent.put(s, env.visited, a, r, a)
            episode_reward += r
            done = False
        # one step is terminated, check if the episode is terminated
        i += 1
        if done:
            env.visited.append(0)
            r = 500
            agent.put(s, env.visited, a, r, 0)
            episode_reward += r
            break

    return env, agent, episode_reward

In [92]:
# TSP 환경 생성
numnodes, maxvalue = 10,10
points = randcoord(numnodes, maxvalue)  # 간단한 예시 좌표
tsp_env = Env(points)

# DQN 에이전트 생성
agent = Agent(states_size=2*tsp_env.n_stops, actions_size=len(points))

# DQN 학습
num_episodes = 800
history = []
for episode in range(num_episodes):
    tsp_env, agent, total_reward = run_episode(tsp_env, agent, episode, verbose=1)
    history.append(total_reward)
    print(f"Episode {episode + 1}, Total Reward: {total_reward}")

# 학습된 에이전트로 최적 경로 예측
def predict_optimal_path(env, agent):
    s = env.reset()
    state = trans(s, [], agent.actions_size)
    path = [s]

    while len(path) < env.n_stops:
        input_data = trans(s, path, agent.actions_size)
        a = agent.act(input_data)
        s_next, _, _ = env.step(s, a)
        path.append(s_next)
        s = s_next
    path.append(s)
    return path

# 최적 경로 및 길이 출력
optimal_path = predict_optimal_path(tsp_env, agent)
optimal_length = sum([tsp_env.dist_matrix[optimal_path[i - 1]][optimal_path[i]] for i in range(1, len(optimal_path))])

print(f"Optimal Path: {optimal_path}")
print(f"Optimal Path Length: {optimal_length}")

permutation, distance = Findoptsol(np.array(tsp_env.dist_matrix))
print(f"real_Optimal Path: {permutation}")
print(f"real_Optimal Path Length: {distance}")

# 1000번까지 에피소드를 진행했으나 적합이 잘 되지 않음

5 -1.9895625089960105 False
3 -4.905091251117594 False
6 -6.75623719662853 False
2 -8.093492715402329 False
1 -7.620784652302808 False
7 -8.63933616755769 False
Episode 1, Total Reward: -2038.004504492005
4 -10.843880223714587 False
3 -6.980965374268762 False
1 -8.062794569850828 False
7 -8.63933616755769 False
6 -9.189377103453513 False
2 -8.093492715402329 False
Episode 2, Total Reward: -2051.8098461542477
5 -1.9895625089960105 False
6 -2.5146136553162997 False
8 -5.004577401674917 False
7 -9.21539248558825 False
1 -8.63933616755769 False
3 -8.062794569850828 False
Episode 3, Total Reward: -2035.426276788984
9 -8.579293986144062 False
3 -5.895412073873364 False
4 -6.980965374268762 False
8 -9.351745191179539 False
7 -9.21539248558825 False
5 -6.890753875802023 False
1 -3.2013328857401957 False
Episode 4, Total Reward: -1550.114895872596
7 -8.756186467218008 False
4 -4.250813147599771 False
8 -9.351745191179539 False
1 -6.742121402603406 False
Episode 5, Total Reward: -3029.1008662086

7 -9.21539248558825 False


3 -7.705775893101598 False
4 -6.980965374268762 False


2 -4.585675628012074 False


5 -5.77799343429354 False
Episode 11, Total Reward: -2038.834847802043
1 -2.2601094764485308 False
4 -11.509995892926003 False
8 -9.351745191179539 False
6 -5.004577401674917 False
7 -9.189377103453513 False
3 -7.705775893101598 False
Episode 12, Total Reward: -2045.0215809587842
3 -6.226340755216833 False
8 -2.6150299035155595 False
2 -8.192416453410834 False
6 -8.093492715402329 False
7 -9.189377103453513 False
9 -2.3673957673885857 False
Episode 13, Total Reward: -2036.6840526983879
8 -4.569044986778715 False
1 -6.742121402603406 False
4 -11.509995892926003 False
3 -6.980965374268762 False
2 -6.841918956903541 False
9 -2.2550833802954076 False
5 -6.591803321334266 False
Episode 14, Total Reward: -1545.4909333151102
5 -1.9895625089960105 False
7 -6.890753875802023 False
3 -7.705775893101598 False
9 -5.895412073873364 False
Episode 15, Total Reward: -3022.481504351773
1 -2.2601094764485308 False
2 -7.620784652302808 False
9 -2.2550833802954076 False
8 -7.853704773177191 False
Episode 

...

6 -5.004577401674917 False
2 -8.093492715402329 False


4 -4.585675628012074 False


1 -11.509995892926003 False


9 -9.070180154761154 False
5 -6.591803321334266 False


Episode 791, Total Reward: -1549.4247701008894
4 -10.843880223714587 False
8 -9.351745191179539 False
2 -8.192416453410834 False
5 -5.77799343429354 False
1 -3.2013328857401957 False
7 -8.63933616755769 False
Episode 792, Total Reward: -2046.0067043558965
9 -8.579293986144062 False
2 -2.2550833802954076 False
8 -8.192416453410834 False
3 -2.6150299035155595 False
Episode 793, Total Reward: -3021.6418237233656
8 -4.569044986778715 False
1 -6.742121402603406 False
6 -2.0415606574273 False
3 -6.75623719662853 False
5 -4.905091251117594 False
4 -8.86012987256621 False
7 -4.250813147599771 False
9 -2.3673957673885857 False
Episode 794, Total Reward: -1040.4923942821101
3 -6.226340755216833 False
8 -2.6150299035155595 False
4 -9.351745191179539 False
2 -4.585675628012074 False
6 -8.093492715402329 False
1 -2.0415606574273 False
9 -9.070180154761154 False
Episode 795, Total Reward: -1541.9840250055147
2 -7.653020649610424 False
1 -7.620784652302808 False
6 -2.0415606574273 False
3 -6.75623719